In [ ]:
## analysing 16s faecal microbiome data from wombat scat samples using Qiime2. (V2024.2)

In [ ]:
## first we will import our data using the import manifest method. 

In [ ]:
## we only imported the R1 sequences following DOI: 10.1128/AEM.00062-07 (figure 1)

In [ ]:
qiime tools import \
--type 'SampleData[SequencesWithQuality]' \
--input-path /hpcfs/users/a1805545/16s_reads/import-manifest-11072024.tsv \
--output-path /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW_SHNW_all_16s.qza \
--input-format SingleEndFastqManifestPhred33V2 

In [ ]:
## once imported we will use the deblur method to generate features (OTU's/ASV's)
## set the trim length 

In [ ]:
qiime deblur denoise-16S \
--i-demultiplexed-seqs /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW_all_16s.qza \
--p-trim-length 150 \
--p-sample-stats \
--o-representative-sequences /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-rep-seqs.qza \
--o-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-table.qza \
--o-stats /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-deblur-stats.qza \
--verbose 

In [ ]:
## then we will visualize the results

In [ ]:
qiime feature-table tabulate-seqs \
--i-data /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-rep-seqs.qza\
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-rep-seqs.qzv

In [ ]:
qiime feature-table summarize \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-table.qza \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-faecal-table.qzv

In [ ]:
## following this we created a phylogenetic tree using SEPP.

In [ ]:
qiime fragment-insertion sepp \
--i-representative-sequences /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-rep-seqs.qza \
--i-reference-database /hpcfs/users/a1805545/16s_reads/classifier/sepp-refs-silva-128.qza \
--o-tree /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-sepp-tree.qza \
--o-placements /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-sepp-placements.qza \
--verbose 


In [ ]:
## next we mapped our reads to the tree and removed any that didnt map.

In [ ]:
qiime fragment-insertion filter-features \
  --i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table.qza \
  --i-tree /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-sepp-tree.qza \
  --o-filtered-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-filtered-table.qza \
  --o-removed-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-removed-table.qza \
  --verbose

In [ ]:
## all features mapped so we will use the unfiltered table.

In [ ]:
## now we will perform an alpha rarefaction to determine an appropriate sampling depth.

In [ ]:
qiime diversity alpha-rarefaction \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--o-visualization NHNW-SHNW-faecal-table-rarefaction.qzv \
--p-max-depth 56000 

In [ ]:
## Looks like 25K is where most samples level out will use that as our sampling depth.

In [ ]:
## Now we will reove samples following the SHNW 16s analysis, soil and NTCs.

In [ ]:
qiime feature-table filter-samples \
--i-table  /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-where "Species IN ('Soil', 'NTC')" \
--p-exclude-ids \
--o-filtered-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil.qza

In [ ]:
## now we will remove any samples that dont have atleast 25K reads. 

In [ ]:
qiime feature-table filter-samples \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-min-frequency 25000 \
--o-filtered-table NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qza

In [1]:
## convert the table to viewable to make sure everything has been removed correctly. 

In [ ]:
qiime feature-table summarize \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qza \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qzv \

In [ ]:
## next we will assign taxonomy using SKlearn and the SILVA database. 

In [ ]:
qiime feature-classifier classify-sklearn \
--i-reads /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-rep-seqs.qza \
--i-classifier /hpcfs/users/a1805545/16s_reads/classifier/silva-139-99-515-806-classifier.qza \
--o-classification /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNE-22-23-SILVA-taxa.qza

In [ ]:
## we will make a barplot with the taxa assignments.

In [ ]:
qiime taxa barplot \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qza \
--i-taxonomy /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-22-23-SILVA-taxa.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-22-23-SILVA-taxa-Barplot-25000-reads.qzv

In [ ]:
## next we will remove duplicates, joey and contaminated samples, according to the SHNW 16s analysis

In [ ]:
qiime feature-table filter-samples \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-faecal-table-no-soil-atleast-25000-reads.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--p-where  "name IN ('B10-SRR12658844', 'B11-SRR12658843', 'B13-SRR12658841', 'B14-SRR12658840', 'B15-SRR12658839', 'B16-SRR12658838', 'B17-SRR12658837', 'M7b-SRR12658823', 'M6-SRR12658826', 'M14-SRR12658813', 'M1b-SRR12658834', 'M1c-SRR12658833', 'M3b-SRR12658830', 'M4b-SRR12658828', 'M9b-SRR12658820')" \
--p-exclude-ids \
--o-filtered-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-atleast-25000-reads.qza

In [3]:
## now we will make new barplots with samples removed.

In [ ]:
qiime taxa barplot \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-atleast-25000-reads.qza \
--i-taxonomy /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-22-23-SILVA-taxa.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-barplot-no-soil-atleast-25000-reads.qzv

In [4]:
# lets collapse the table into populations to get an idea of larger trends. 

In [ ]:
qiime feature-table group \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-no-soil-atleast-25000-reads.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--m-metadata-column cluster \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-cluster.qza

In [ ]:
## Make barplots of the clusters - note 14/08/2024 remake metadata file so it corresponds to the condensed table.

In [ ]:
qiime taxa barplot \
--i-table /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-faecal-table-cluster.qza \
--i-taxonomy /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-22-23-SILVA-taxa.qza \
--m-metadata-file /hpcfs/users/a1805545/16s_reads/metadata.txt \
--o-visualization /hpcfs/users/a1805545/16s_reads/qiime_output/NHNW-SHNW-final-cluster-faecal-barplot-no-soil-atleast-25000-reads.qzv